In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import gmaps
import requests as rq

# Import API key
import config

# Configure gmaps
gkey = config.gkey
gmaps.configure(api_key=gkey)

In [10]:
# Create aiport dataframe
cities_df = pd.read_csv('Resources//weather_data.csv')
cities_df.dropna(inplace = True) 
cities_df.head()

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City DateTime
0,Torbay,47.67,-52.73,44.01,74,75,16.11,CA,2020-11-13 10:41:31
1,Hermanus,-34.42,19.23,66.00,79,91,5.01,ZA,2020-11-13 10:40:52
2,Yellowknife,62.46,-114.35,8.60,92,75,8.05,CA,2020-11-13 10:42:21
3,Cape Town,-33.93,18.42,75.00,68,0,17.22,ZA,2020-11-13 10:41:38
4,Grindavik,63.84,-22.43,37.40,86,75,17.22,IS,2020-11-13 10:41:49


In [12]:
# Store latitude and longitude in locations
locations = cities_df[["City Latitude", "City Longitude"]]
humidity = cities_df["Humidity"]

In [51]:
zero_coor = (33.93,18.42)
figure_layout = {'width': '1050px', 'margin': '0 auto 0 auto'}
# Plot Heatmap
fig = gmaps.figure(center=zero_coor,zoom_level=2,layout=figure_layout,map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=2.0)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='1050px'))

In [46]:
filtered_temp_df = cities_df.loc[(cities_df['Max Temperature'] < 90) & (cities_df['Max Temperature'] >= 65) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloud Coverage'] < 20)]
filtered_temp_df.describe()

,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed
count,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000
mean,4.240492,24.346230,76.316393,60.163934,3.245902,5.639672
std,22.263260,78.787535,6.769721,21.181108,5.058510,2.261419
min,-34.040000,-151.750000,65.370000,13.000000,0.000000,1.010000
25%,-19.980000,-15.540000,69.800000,48.000000,0.000000,3.710000
50%,12.220000,40.210000,76.440000,62.000000,0.000000,5.820000
75%,21.640000,59.530000,81.090000,78.000000,4.000000,7.290000
max,36.370000,153.570000,88.700000,94.000000,19.000000,9.800000


In [50]:
filtered_temp_df.head()

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City DateTime
5,Kruisfontein,-34.00,24.73,80.01,60,7,5.99,ZA,2020-11-13 10:41:28
26,Guerrero Negro,27.98,-114.06,68.18,59,0,8.59,MX,2020-11-13 10:43:14
28,Tabou,4.42,-7.35,81.09,80,12,6.26,CI,2020-11-13 10:43:18
63,Saint-Philippe,-21.36,55.77,77.00,78,0,3.36,RE,2020-11-13 10:41:48
69,Porbandar,21.64,69.61,80.08,43,0,9.19,IN,2020-11-13 10:44:48


In [ ]:
for city in filtered_temp_df: 
    lat = filtered_temp_df.loc[filtered_temp_df['City'] == city,'City Latitude'] 
    lng = filtered_temp_df.loc[filtered_temp_df['City'] == city, 'City Longitude']  
    criteria = "Hotel"
    coords = (lat, lng)
    radius = 5000

    params = {
    "location": coords,
    "keyword": criteria,
    "radius": radius,
    "key": gkey
    }
    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [ ]:
# Plot Heatmap
fig = gmaps.figure(center=zero_coor,zoom_level=2,layout=figure_layout,map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=2.0)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig